In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb

In [ ]:
# Import raw data
calander = pd.read_csv('data/calendar.csv')
df=pd.read_csv('data/sales_train_validation.csv')
sell_prices = pd.read_csv('data/sell_prices.csv')

In [ ]:
calander.head()

In [ ]:
df.head()

In [ ]:
len(df) * 1913

In [ ]:
df.head()

In [ ]:
# Before one row represents all sales for one item (30490 rows)
# After one row represents one day's sales for one item (58327370 rows = 30490 * 1913)
cat_cols = ['id','item_id','dept_id','cat_id','store_id','state_id']
numcols = [f"d_{day}" for day in range(1,1914)]
df = pd.melt(df, id_vars=cat_cols, value_vars = numcols, var_name='d',value_name='sales')

In [ ]:
df.head()

In [ ]:
calander.head()

In [ ]:
# Merge with calander
df_temp = df.copy()
calander_temp = calander.copy()[:10]
calander_temp.merge(df_temp, how='left',left_on='d',right_on='d')

In [ ]:
def create_train_data(ds=2011,de=2016):
    # Import raw data
    calander = pd.read_csv('data/calendar.csv')
    df=pd.read_csv('data/sales_train_validation.csv')
    sell_prices = pd.read_csv('data/sell_prices.csv')
    calander = calander[(calander['year'] >= ds) & (calander['year'] <de)]
    # Before one row represents all sales for one item (30490 rows)
    # After one row represents one day's sales for one item (58327370 rows = 30490 * 1913)
    cat_cols = ['id','item_id','dept_id','cat_id','store_id','state_id']
    numcols = [f"d_{day}" for day in range(1,1914)]
    df = pd.melt(df, id_vars=cat_cols, value_vars = numcols, var_name='d',value_name='sales')
    
    # Merge with calander
    df = df.merge(calander,on='d')
    return df

In [ ]:
train_val_df = create_train_data(ds=2011,de=2012)

In [ ]:
train_val_df.year.unique()

In [ ]:
train_val_df.to_csv('train_val_df_2011.csv')

In [2]:
train_df = pd.read_csv('train_val_df_2011.csv')

C:\Users\nickh\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (15,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [18]:
def create_train_data(df_train):
    features = ['item_id','dept_id','cat_id','store_id','state_id','wday','month','year']
    cat = ['item_id','dept_id','cat_id','store_id','state_id']
    target = 'sales'
    for c in cat:
        df_train[c] = df_train[c].astype('category').cat.codes
        
    X = train_df[features][:-28].values
    X_val = train_df[features][-28:].values
    y=train_df[target][:-28].values
    y_val=train_df[target][-28:].values
    train_data = lgb.Dataset(X,label=y,feature_name = features, categorical_feature = cat)
    val_data = lgb.Dataset(X_val,label=y_val,feature_name = features, categorical_feature= cat)
    
    return train_data, val_data

In [3]:
cat = ['item_id','dept_id','cat_id','store_id','state_id']
a= train_df['item_id'].str.split('_')

In [6]:
sub = train_df[:20]

In [14]:
test_df = sub.copy()
for c in cat:
    cat_codename = c + "_code"    
    test_df[cat_codename] = test_df[c].astype('category').cat.codes


In [15]:
test_df

,Unnamed: 0,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,...,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,item_id_code,dept_id_code,cat_id_code,store_id_code,state_id_code
0,0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,...,NaN,NaN,0,0,0,0,0,0,0,0
1,1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,...,NaN,NaN,0,0,0,1,0,0,0,0
2,2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,...,NaN,NaN,0,0,0,2,0,0,0,0
3,3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,...,NaN,NaN,0,0,0,3,0,0,0,0
4,4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,...,NaN,NaN,0,0,0,4,0,0,0,0
5,5,HOBBIES_1_006_CA_1_validation,HOBBIES_1_006,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,...,NaN,NaN,0,0,0,5,0,0,0,0
6,6,HOBBIES_1_007_CA_1_validation,HOBBIES_1_007,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,...,NaN,NaN,0,0,0,6,0,0,0,0
7,7,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1,12,2011-01-29,...,NaN,NaN,0,0,0,7,0,0,0,0
8,8,HOBBIES_1_009_CA_1_validation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,d_1,2,2011-01-29,...,NaN,NaN,0,0,0,8,0,0,0,0
9,9,HOBBIES_1_010_CA_1_validation,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,...,NaN,NaN,0,0,0,9,0,0,0,0


In [19]:
train_data,val_data = create_train_data(train_df)

### Example data and label example

In [ ]:
data = np.random.rand(500, 10)  # 500 entities, each contains 10 features
label = np.random.randint(2, size=500)  # binary target

In [ ]:
data

In [ ]:
label